## Метод shift. Сдвиг данных (с опережением и с запаздыванием)

Под «сдвигом» понимается перемещение данных назад и вперед по временной оси. У  объектов Series и  DataFrame имеется метод shift для «наивного»
сдвига в обе стороны без модификации индекса:

In [1]:
import pandas as pd
import numpy as np

In [2]:
ts = pd.Series(np.random.standard_normal(4), index=pd.date_range('2024-01-01', periods=4, freq='ME'))
ts

2024-01-31   -0.861688
2024-02-29    0.143626
2024-03-31    1.115996
2024-04-30    3.599446
Freq: ME, dtype: float64

In [3]:
ts.shift(2)

2024-01-31         NaN
2024-02-29         NaN
2024-03-31   -0.861688
2024-04-30    0.143626
Freq: ME, dtype: float64

In [4]:
ts.shift(-2)

2024-01-31    1.115996
2024-02-29    3.599446
2024-03-31         NaN
2024-04-30         NaN
Freq: ME, dtype: float64

Поскольку наивный сдвиг не изменяет индекс, некоторые данные отбрасываются. Но  если известна частота, то ее можно передать методу shift, чтобы сдвинуть вперед временные метки, а не сами данные:

In [5]:
 ts.shift(2, freq='ME')

2024-03-31   -0.861688
2024-04-30    0.143626
2024-05-31    1.115996
2024-06-30    3.599446
Freq: ME, dtype: float64

Можно указывать и другие частоты, что позволяет гибко смещать данные в прошлое и в будущее:

In [6]:
ts.shift(3, freq='D')

2024-02-03   -0.861688
2024-03-03    0.143626
2024-04-03    1.115996
2024-05-03    3.599446
dtype: float64

In [7]:
ts.shift(1, freq='90min')

2024-01-31 01:30:00   -0.861688
2024-02-29 01:30:00    0.143626
2024-03-31 01:30:00    1.115996
2024-04-30 01:30:00    3.599446
dtype: float64

Здесь min обозначает минуты. Отметим, что параметр freq задает смещение, применяемое к временным меткам, но истинная частота данных (если таковая присутствует) при этом не изменяется.

## Метод asfreq. Преобразование частоты периода

Периоды и  объекты PeriodIndex можно преобразовать с  изменением частоты,
воспользовавшись методом asfreq. Для примера предположим, что имеется годовой период, который мы хотим преобразовать в месячный, начинающийся или заканчивающийся на границе года. Это довольно просто:

In [8]:
p = pd.Period('2024', freq='Y-DEC')
p

Period('2024', 'Y-DEC')

In [9]:
p.asfreq('M', how='start')

Period('2024-01', 'M')

In [10]:
p.asfreq('M', how='end')

Period('2024-12', 'M')

In [11]:
p.asfreq('M')

Period('2024-12', 'M')

## Преобразование временных меток в периоды и обратно

Объекты Series и DataFrame, индексированные временными метками, можно преобразовать в периоды методом to_period:

In [12]:
dates = pd.date_range('2024-01-01', periods=3, freq='ME')
ts = pd.Series(np.random.standard_normal(3), index=dates)
ts

2024-01-31   -0.679901
2024-02-29    1.743016
2024-03-31    0.205783
Freq: ME, dtype: float64

In [13]:
pts = ts.to_period()
pts

2024-01   -0.679901
2024-02    1.743016
2024-03    0.205783
Freq: M, dtype: float64

Поскольку периоды всегда ссылаются на непересекающиеся временные промежутки, то временная метка может принадлежать только одному периоду данной частоты. Частота нового объекта PeriodIndex по умолчанию выводится из временных меток, но можно задать любую поддерживаемую частоту. Наличие повторяющихся периодов в результате также не приводит ни к каким проблемам:

In [14]:
dates = pd.date_range('2024-01-29', periods=6)
ts2 = pd.Series(np.random.standard_normal(6), index=dates)
ts2

2024-01-29    0.188176
2024-01-30   -0.681875
2024-01-31   -0.681815
2024-02-01    0.417289
2024-02-02    0.327378
2024-02-03   -0.643967
Freq: D, dtype: float64

In [15]:
ts2.to_period('M')

2024-01    0.188176
2024-01   -0.681875
2024-01   -0.681815
2024-02    0.417289
2024-02    0.327378
2024-02   -0.643967
Freq: M, dtype: float64

In [16]:
pts2 = ts2.to_period()
pts2

2024-01-29    0.188176
2024-01-30   -0.681875
2024-01-31   -0.681815
2024-02-01    0.417289
2024-02-02    0.327378
2024-02-03   -0.643967
Freq: D, dtype: float64

Для обратного преобразования во временные метки служит метод to_timestamp, который возвращает объект типа DatetimeIndex:

In [17]:
pts2.to_timestamp(how='end')

2024-01-29 23:59:59.999999999    0.188176
2024-01-30 23:59:59.999999999   -0.681875
2024-01-31 23:59:59.999999999   -0.681815
2024-02-01 23:59:59.999999999    0.417289
2024-02-02 23:59:59.999999999    0.327378
2024-02-03 23:59:59.999999999   -0.643967
Freq: D, dtype: float64

In [18]:
pts2.to_timestamp(how='start')

2024-01-29    0.188176
2024-01-30   -0.681875
2024-01-31   -0.681815
2024-02-01    0.417289
2024-02-02    0.327378
2024-02-03   -0.643967
Freq: D, dtype: float64